In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
poke=pd.read_csv("/Users/Shuxian/Documents/CSYE7200-Fall2016/predictemall/poke/poke_us.csv",header=0,low_memory=False)

In [3]:
poke=poke[poke['pokestopDistanceKm'] !="?"]

In [6]:
train,test=train_test_split(poke,test_size=0.2, random_state=0)

In [7]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 10), random_state=1)

In [12]:
X_train=train[train.columns[1:]]
y_train=train[train.columns[0]]

In [15]:
X_test=test[test.columns[1:]]
y_test=test[test.columns[0]]

In [14]:
clf.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [16]:
y_pred=clf.predict(X_test)

In [19]:
accuracy_score(y_test, y_pred)

0.17499481650425047

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [27]:
scores = cross_val_score(clf, X_train, y_train)

/Users/Shuxian/anaconda/lib/python3.5/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [25]:
scores

array([ 0.1747205 ,  0.17496112,  0.17517907])

In [32]:
from sklearn.multiclass import OneVsRestClassifier

In [35]:
ovall=OneVsRestClassifier(clf).fit(X_train, y_train)

In [36]:
y_pred=ovall.predict(X_test)

In [37]:
accuracy_score(y_test, y_pred)

0.16852581380883266

# Optimization "RARITY and TYPE"

## Rarity

In [11]:
poke=pd.read_csv("/Users/Shuxian/Documents/CSYE7200-Fall2016/predictemall/poke/poke.csv",header=0,low_memory=False)
poke=poke[poke['pokestopDistanceKm'] !="?"]

In [12]:
rareId = [1,2,4,5,7,8,9,12,15,18,22,24,25,28,30,33,36,38,40,44,45,47,49,51,53,55,57,58,61,
          63,64,67,70,73,75,78,80,82,83,85,87,88,91,93,94,95,97,99,101,103,105,106,107,110,111,117,119,121,123,124,
          125,126,127,128,133,138,140,147]
          
veryRareId = [3,6,9,26,31,34,59,62,65,68,71,76,83,89,108,112,113,114,115,122,130,131,132,134,135,136,137,139,141,142
              ,143,144,145,146,148,148,149,150,151]

In [13]:
rare=poke.loc[poke['class'].isin(rareId)]
rare['rarity']='1'

/Users/Shuxian/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [14]:
veryrare=poke.loc[poke['class'].isin(veryRareId)]
veryrare['rarity']='2'

/Users/Shuxian/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [15]:
uncommonId=rareId+veryRareId

In [17]:
len(veryRareId)

39

In [18]:
len(rareId)

68

uncommon=poke.loc[poke['class'].isin(uncommonId)]

uncommon.to_csv("/Users/Shuxian/Documents/CSYE7200-Fall2016/predictemall/poke/uncommon_poke.csv",index=False)

In [19]:
commonId=list(set(poke['class'].unique())-set(uncommonId))

In [20]:
common=poke.loc[poke['class'].isin(commonId)]
common['rarity']='0'

/Users/Shuxian/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [21]:
# concatenate rare, veryrare and common
newpoke=pd.concat([rare,veryrare,common])

In [22]:
# randomize data
newpoke=newpoke.iloc[np.random.permutation(len(newpoke))].reset_index(drop=True)

### use pokemonGo data to find pokemon main type

In [26]:
poketype=pd.read_csv("/Users/Shuxian/Documents/CSYE7200-Fall2016/predictemall/pokemonGO.csv",header=0)

In [27]:
poketype.set_index(poketype['Pokemon No.'], drop=True, append=False, inplace=True, verify_integrity=True)

In [28]:
poketype.drop(poketype.columns[[0,1,3,4,5,6]],axis=1,inplace=True)

In [29]:
pokedict=poketype.to_dict(orient='dict')['Type 1']

In [31]:
type=[]
for x in newpoke['class']:
    type.append(pokedict.get(x))

In [32]:
newpoke['type']=type

newpoke.drop('type',axis=1,inplace=True)

In [34]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(newpoke.type.unique())
list(le.classes_)
newpoke['type']=le.transform(newpoke['type'])

In [39]:
newpoke.head(5)

,class,latitude,longitude,appearedTimeOfDay,appearedHour,appearedMinute,appearedDayOfWeek,appearedDay,terrainType,closeToWater,...,cooc_144,cooc_145,cooc_146,cooc_147,cooc_148,cooc_149,cooc_150,cooc_151,rarity,type
0,21,53.602550,-113.444897,4,23,33,4,4,13,0,...,0,0,0,0,0,0,0,0,0,10
1,19,42.324002,-71.578191,4,3,24,7,7,5,0,...,0,0,0,0,0,0,0,0,0,10
2,129,-20.469650,-54.592661,2,16,41,4,4,13,0,...,0,0,0,0,0,0,0,0,0,14
3,41,43.710060,-79.395080,4,23,9,5,5,13,1,...,0,0,0,0,0,0,0,0,0,11
4,4,33.756573,-84.389152,4,0,1,6,6,13,0,...,0,0,0,0,0,0,0,0,1,5


In [36]:
newpoke.to_csv("/Users/Shuxian/Documents/CSYE7200-Fall2016/predictemall/poke/poke_rarity_type.csv",index=False)

### trian with very rare

In [102]:
train,test=train_test_split(veryrare,test_size=0.2, random_state=0)

In [103]:
X_train=train[train.columns[1:]]
y_train=train[train.columns[0]]
X_test=test[test.columns[1:]]
y_test=test[test.columns[0]]

In [104]:
clf.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [105]:
y_pred=clf.predict(X_test)

In [106]:
accuracy_score(y_test, y_pred)

0.41830065359477125